# 📚 Kafka Integration Example (Producer & Consumer)

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
import cv2, base64, json, numpy as np, tensorflow as tf

In [ ]:
# Kafka Config
TOPIC = 'image-topic'
SERVER = 'localhost:9092'

In [ ]:
# --- Producer (simulates webcam)
producer = KafkaProducer(bootstrap_servers=[SERVER],
                         value_serializer=lambda x: json.dumps(x).encode())

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    _, buffer = cv2.imencode('.jpg', frame)
    img_bytes = base64.b64encode(buffer).decode()
    producer.send(TOPIC, value={"image": img_bytes})
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()

# --- Consumer (shows image)
consumer = KafkaConsumer(TOPIC,
                         bootstrap_servers=[SERVER],
                         auto_offset_reset='latest',
                         value_deserializer=lambda x: json.loads(x.decode()))

for msg in consumer:
    img_data = base64.b64decode(msg.value['image'])
    arr = np.frombuffer(img_data, np.uint8)
    frame = cv2.imdecode(arr, 1)
    cv2.imshow("Kafka Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()